In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# conda install pytorch torchvision -c pytorch 
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut

# import MLPRegressor
# XGBRegressor
# LGBMRegressor
# CatBoostRegressor
# KNeighborsRegressor
# Lasso
# Ridge
# ElasticNet
# SGDRegressor

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor


In [19]:
# data/output_df의 파일들과 data/weather의 파일들을 읽어온다.
seoul_output_df = pd.read_csv('data/output_df/seoul_spring_df.csv')
seoul_spring_df = pd.read_csv('data/weather/seoul_spring.csv')

# 위에 파일들을 합친다.
seoul_df = pd.merge(seoul_output_df, seoul_spring_df, on='일시', how='left')

# 출력
seoul_df

,일시,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,2000,0,0,6.3,18.9,-4.8,3.1,389.50
1,2000,0,0,11.9,23.5,1.8,30.7,440.90
2,2000,0,0,17.5,30.5,7.9,75.2,443.69
3,2001,0,0,5.0,21.0,-6.1,18.1,396.53
4,2001,0,0,13.6,28.2,2.1,12.3,492.04
...,...,...,...,...,...,...,...,...
61,2020,0,1,11.1,23.8,1.9,16.9,637.52
62,2020,0,1,18.0,30.0,9.2,112.4,564.83
63,2021,5,228,9.0,22.9,-1.2,110.9,476.94
64,2021,5,228,14.2,28.2,3.1,124.1,558.28


In [20]:
# 일반봄배추:면적 (ha)	생산량 (톤)	평균기온(°C) 컬럼만 추출
seoul_df = seoul_df[['일반봄배추:면적 (ha)', '생산량 (톤)', '평균기온(°C)']]
seoul_df

,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C)
0,0,0,6.3
1,0,0,11.9
2,0,0,17.5
3,0,0,5.0
4,0,0,13.6
...,...,...,...
61,0,1,11.1
62,0,1,18.0
63,5,228,9.0
64,5,228,14.2


In [21]:
# 입력데이터 정규화 (MinMaxScaler)
scaler = MinMaxScaler()
scaler.fit(seoul_df.iloc[:, 1:])
seoul_df.iloc[:, 1:] = scaler.transform(seoul_df.iloc[:, 1:])
seoul_df





,일반봄배추:면적 (ha),생산량 (톤),평균기온(°C)
0,0,0.000000,0.167702
1,0,0.000000,0.515528
2,0,0.000000,0.863354
3,0,0.000000,0.086957
4,0,0.000000,0.621118
...,...,...,...
61,0,0.004386,0.465839
62,0,0.004386,0.894410
63,5,1.000000,0.335404
64,5,1.000000,0.658385


In [22]:
# # 입력데이터 정규화 (StandardScaler)
# scaler = StandardScaler()
# scaler.fit(seoul_df.iloc[:, 1:])
# seoul_df.iloc[:, 1:] = scaler.transform(seoul_df.iloc[:, 1:])
# seoul_df
